# 📈 Google Stock Price Prediction using LSTM

This notebook demonstrates how to use an LSTM neural network to predict Google (GOOG) stock prices using historical data.

**Author**: Palak Verma

Dataset Source: [Yahoo Finance](https://finance.yahoo.com/)

In [ ]:
!pip install yfinance pandas numpy matplotlib scikit-learn tensorflow

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [ ]:
# Download Google stock data
df = yf.download('GOOG', start='2010-01-01', end='2023-11-01')
df = df[['Close']]
df.head()


In [ ]:
# Scaling and sequence creation
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

sequence_length = 60
X, y = [], []

for i in range(sequence_length, len(scaled_data)):
    X.append(scaled_data[i-sequence_length:i])
    y.append(scaled_data[i])

X, y = np.array(X), np.array(y)

# Split into training and testing
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=20, batch_size=32)


In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
real = scaler.inverse_transform(y_test.reshape(-1, 1))

plt.figure(figsize=(12,6))
plt.plot(real, color='blue', label='Actual Google Stock Price')
plt.plot(predictions, color='red', label='Predicted Google Stock Price')
plt.title('Google Stock Price Prediction with LSTM')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
